In [32]:
from pyspark.sql import SparkSession
spark= SparkSession.builder.appName("Cars").getOrCreate()

In [33]:
df=spark.read.csv(r"F:/desktop/Spark/datasets/car_evaluation.csv",header=True,inferSchema=True).toDF(
  'buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class')
df.show()

+------+-----+-----+-------+--------+------+-----+
|buying|maint|doors|persons|lug_boot|safety|class|
+------+-----+-----+-------+--------+------+-----+
| vhigh|vhigh|    2|      2|   small|   med|unacc|
| vhigh|vhigh|    2|      2|   small|  high|unacc|
| vhigh|vhigh|    2|      2|     med|   low|unacc|
| vhigh|vhigh|    2|      2|     med|   med|unacc|
| vhigh|vhigh|    2|      2|     med|  high|unacc|
| vhigh|vhigh|    2|      2|     big|   low|unacc|
| vhigh|vhigh|    2|      2|     big|   med|unacc|
| vhigh|vhigh|    2|      2|     big|  high|unacc|
| vhigh|vhigh|    2|      4|   small|   low|unacc|
| vhigh|vhigh|    2|      4|   small|   med|unacc|
| vhigh|vhigh|    2|      4|   small|  high|unacc|
| vhigh|vhigh|    2|      4|     med|   low|unacc|
| vhigh|vhigh|    2|      4|     med|   med|unacc|
| vhigh|vhigh|    2|      4|     med|  high|unacc|
| vhigh|vhigh|    2|      4|     big|   low|unacc|
| vhigh|vhigh|    2|      4|     big|   med|unacc|
| vhigh|vhigh|    2|      4|   

In [34]:
df.printSchema()

root
 |-- buying: string (nullable = true)
 |-- maint: string (nullable = true)
 |-- doors: string (nullable = true)
 |-- persons: string (nullable = true)
 |-- lug_boot: string (nullable = true)
 |-- safety: string (nullable = true)
 |-- class: string (nullable = true)



In [35]:
df.columns

['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']

In [43]:
from pyspark.sql.functions import regexp_replace
val=['vhigh','high','low','5more','more','small','big']
labels=['unacc','acc','good','vgood']
att=[val,labels]
for col in df.columns[:-1]:
    for i in range(len(val)):
        if val[i] in list (df.select(col).toPandas()[col]):
            df=df.withColumn(col,regexp_replace(df[col],val[i],str(i)))
for cols in df.columns[:-1]:
    df=df.withColumn(cols,df[cols].cast('Integer'))

In [44]:
df.printSchema()

root
 |-- buying: integer (nullable = true)
 |-- maint: integer (nullable = true)
 |-- doors: integer (nullable = true)
 |-- persons: integer (nullable = true)
 |-- lug_boot: integer (nullable = true)
 |-- safety: integer (nullable = true)
 |-- class: string (nullable = true)



In [18]:
#from pyspark.ml.feature import StringIndexer
#indexer=StringIndexer(inputCol='buying',outputCol='buying_index')
#df1=indexer.fit(df).transform(df)
#indexer=StringIndexer(inputCol='maint',outputCol='maint_index')
#df1=indexer.fit(df1).transform(df1)
#indexer=StringIndexer(inputCol='doors',outputCol='doors_index')
#df1=indexer.fit(df1).transform(df1)
#indexer=StringIndexer(inputCol='persons',outputCol='persons_index')
#df1=indexer.fit(df1).transform(df1)
#indexer=StringIndexer(inputCol='lug_boot',outputCol='lug_boot_index')
#df1=indexer.fit(df1).transform(df1)
#indexer=StringIndexer(inputCol='safety',outputCol='safety_index')
#df1=indexer.fit(df1).transform(df1)
#indexer=StringIndexer(inputCol='class',outputCol='class_index')
#df1=indexer.fit(df1).transform(df1)
#df1.show()

+------+-----+-----+-------+--------+------+-----+------------+-----------+-----------+-------------+--------------+------------+-----------+
|buying|maint|doors|persons|lug_boot|safety|class|buying_index|maint_index|doors_index|persons_index|lug_boot_index|safety_index|class_index|
+------+-----+-----+-------+--------+------+-----+------------+-----------+-----------+-------------+--------------+------------+-----------+
| vhigh|vhigh|    2|      2|   small|   med|unacc|         3.0|        3.0|        3.0|          2.0|           2.0|         1.0|        0.0|
| vhigh|vhigh|    2|      2|   small|  high|unacc|         3.0|        3.0|        3.0|          2.0|           2.0|         0.0|        0.0|
| vhigh|vhigh|    2|      2|     med|   low|unacc|         3.0|        3.0|        3.0|          2.0|           1.0|         2.0|        0.0|
| vhigh|vhigh|    2|      2|     med|   med|unacc|         3.0|        3.0|        3.0|          2.0|           1.0|         1.0|        0.0|
| vhig

In [21]:
from pyspark.ml.feature import VectorAssembler
va=VectorAssembler(inputCols=['buying_index', 'maint_index', 'doors_index', 'persons_index', 'lug_boot_index', 'safety_index'],outputCol='features')
output=va.transform(df1)
output.show()

+------+-----+-----+-------+--------+------+-----+------------+-----------+-----------+-------------+--------------+------------+-----------+--------------------+
|buying|maint|doors|persons|lug_boot|safety|class|buying_index|maint_index|doors_index|persons_index|lug_boot_index|safety_index|class_index|            features|
+------+-----+-----+-------+--------+------+-----+------------+-----------+-----------+-------------+--------------+------------+-----------+--------------------+
| vhigh|vhigh|    2|      2|   small|   med|unacc|         3.0|        3.0|        3.0|          2.0|           2.0|         1.0|        0.0|[3.0,3.0,3.0,2.0,...|
| vhigh|vhigh|    2|      2|   small|  high|unacc|         3.0|        3.0|        3.0|          2.0|           2.0|         0.0|        0.0|[3.0,3.0,3.0,2.0,...|
| vhigh|vhigh|    2|      2|     med|   low|unacc|         3.0|        3.0|        3.0|          2.0|           1.0|         2.0|        0.0|[3.0,3.0,3.0,2.0,...|
| vhigh|vhigh|    2|  

In [24]:
df1=output.select('features','class_index')
df1.show()

+--------------------+-----------+
|            features|class_index|
+--------------------+-----------+
|[3.0,3.0,3.0,2.0,...|        0.0|
|[3.0,3.0,3.0,2.0,...|        0.0|
|[3.0,3.0,3.0,2.0,...|        0.0|
|[3.0,3.0,3.0,2.0,...|        0.0|
|[3.0,3.0,3.0,2.0,...|        0.0|
|[3.0,3.0,3.0,2.0,...|        0.0|
|[3.0,3.0,3.0,2.0,...|        0.0|
|[3.0,3.0,3.0,2.0,...|        0.0|
|[3.0,3.0,3.0,0.0,...|        0.0|
|[3.0,3.0,3.0,0.0,...|        0.0|
|[3.0,3.0,3.0,0.0,...|        0.0|
|[3.0,3.0,3.0,0.0,...|        0.0|
|[3.0,3.0,3.0,0.0,...|        0.0|
|[3.0,3.0,3.0,0.0,...|        0.0|
|[3.0,3.0,3.0,0.0,...|        0.0|
|[3.0,3.0,3.0,0.0,...|        0.0|
|[3.0,3.0,3.0,0.0,...|        0.0|
|[3.0,3.0,3.0,1.0,...|        0.0|
|[3.0,3.0,3.0,1.0,...|        0.0|
|[3.0,3.0,3.0,1.0,...|        0.0|
+--------------------+-----------+
only showing top 20 rows



In [25]:
(train,test)=df1.randomSplit([0.8,0.2])

In [26]:
from pyspark.ml.classification import DecisionTreeClassifier
clf=DecisionTreeClassifier(featuresCol='features',labelCol='class_index')
clf=clf.fit(train)

In [27]:
pred=clf.transform(test)
pred.show(3)

+-------------------+-----------+--------------------+--------------------+----------+
|           features|class_index|       rawPrediction|         probability|prediction|
+-------------------+-----------+--------------------+--------------------+----------+
|(6,[0,1],[1.0,1.0])|        3.0| [4.0,9.0,25.0,41.0]|[0.05063291139240...|       3.0|
|(6,[0,2],[1.0,1.0])|        3.0|[29.0,114.0,0.0,1...|[0.18954248366013...|       1.0|
|(6,[0,3],[2.0,1.0])|        1.0|[29.0,114.0,0.0,1...|[0.18954248366013...|       1.0|
+-------------------+-----------+--------------------+--------------------+----------+
only showing top 3 rows



In [31]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from sklearn.metrics import confusion_matrix
evaluator= MulticlassClassificationEvaluator(predictionCol='prediction',labelCol='class_index',
                                             metricName='accuracy')

                                             
acc = evaluator. evaluate(pred)
print ('Prediction Accuracy:', acc)

Prediction Accuracy: 0.8826979472140762
